In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df =  pd.read_csv("data/car_price.csv")
train_data = df.loc[:6500,:]
test_data = df.loc[6500:,:]
train_data.set_index("name",inplace=True)
test_data.set_index("name",inplace=True)
train_data.head()

,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
name,,,,,,,,,,,,
Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [16]:
output = train_data.selling_price
train_data.drop("selling_price",inplace=True,axis=1)
train_data = train_data.reset_index()
train_data = train_data.to_numpy()
output = output.to_numpy()
m = output.shape
train_data = np.append(train_data,np.ones([m[0],1]),axis=1)

In [17]:
print(pd.DataFrame(train_data))

                 0       1         2    3      4       5       6    7    8   \
0     580881.772727  2014.0  145500.0  8.0  23.40  1248.0   74.00  5.0  0.0   
1     490000.000000  2014.0  120000.0  4.0  21.14  1498.0  103.52  5.0  0.0   
2     167666.666667  2006.0  140000.0  3.0  17.70  1497.0   78.00  5.0  0.0   
3     225000.000000  2010.0  127000.0  8.0  23.00  1396.0   90.00  5.0  0.0   
4     130000.000000  2007.0  120000.0  8.0  16.10  1298.0   88.20  5.0  0.0   
...             ...     ...       ...  ...    ...     ...     ...  ...  ...   
6496  610000.000000  2015.0   33000.0  8.0  20.73  1373.0   91.10  5.0  0.0   
6497  979000.000000  2015.0   50000.0  8.0  19.67  1582.0  126.20  5.0  0.0   
6498  475571.285714  2012.0  120000.0  4.0  12.05  2179.0  120.00  9.0  0.0   
6499  527500.000000  2018.0   35000.0  8.0  25.40  1186.0   73.97  5.0  0.0   
6500  700000.000000  2018.0   43000.0  8.0  27.40  1498.0   98.63  5.0  0.0   

       9    10   11   12   13      14      15      

In [18]:
Xt = train_data.transpose()
theta = np.dot(np.linalg.inv(np.dot(Xt,train_data)),np.dot(Xt,output))


[ 9.83793362e-01  4.23050560e+03 -1.94531682e-01  3.55826936e+03
 -1.57770379e+03  1.50268497e+01  1.18529706e+02 -5.02233774e+03
 -2.64943505e+04 -2.81319762e+04 -2.58132832e+04  1.88616770e+04
  1.36491066e+04  6.46398140e+03 -7.51155779e+01 -1.37914354e+00
 -8.23268581e+00 -8.44444174e+06]


In [19]:
theta.shape

(18,)

In [3]:
models = train_data.index.unique()
model_mean = []
for n in models:
    model_mean.append(train_data.loc[n,"selling_price"].mean())
name_mapper = dict(zip(models,model_mean))

In [4]:
def clean_name(df):
    df.index = df.index.map(name_mapper)
    return df

In [5]:
def clean_owner(df):
    owner_mapper = {"First Owner":8,"Test Drive Car":4.5,"Second Owner":4,"Third Owner":3,"Fourth & Above Owner":2}
    df.owner = df.owner.map(owner_mapper)
    return df

In [6]:
def one_hot_encoding(df):
    df = pd.get_dummies(df,columns = ["fuel","seller_type","transmission"])
    drop_columns = ["fuel_LPG","seller_type_Trustmark Dealer","transmission_Automatic"]
    df.drop(drop_columns,inplace=True,axis=1)
    return df

In [7]:
def remove_units(df):
    df["mileage"] = df["mileage"].str.extract(r'(\d+.?\d*)')
    df["engine"] = df["engine"].str.extract(r'(\d+)')
    df["max_power"] = df["max_power"].str.extract(r'(\d+.?\d*)')
    return df

In [8]:
def clean_torque(df):
    df["torque_1"] = df["torque"].str.extract(r'(\d+\.?\d*)')
    rpm = df["torque"].str.extract(r'(at|@)\s(\d+,?\d+)-?(\d*)')
    for index,row in rpm.iterrows():
        if row[2] == '':
            row[2] = row[1]
    df["lower_rpm"] = rpm[1]
    df["upper_rpm"] = rpm[2]
    df["lower_rpm"].replace(to_replace='(\d+),(\d*)',value=r"\1\2",inplace=True,regex=True)
    df["upper_rpm"].replace(to_replace='(\d+),(\d*)',value=r"\1\2",inplace=True,regex=True)
    df.drop("torque",inplace=True,axis=1)
    return df

In [9]:
def clean_car_data(df):
    df = clean_name(df)
    df = one_hot_encoding(df)
    df = clean_owner(df)
    df = remove_units(df)
    df = clean_torque(df)
    df = df.astype(float)
    df.fillna(df.mean(),inplace=True)
    y = df.selling_price
    df.drop("selling_price",inplace=True,axis=1)
    df = df.reset_index()
    df = df.to_numpy()
    y = y.to_numpy()
    m = y.shape
    df = np.append(df,np.ones([m[0],1]),axis=1)
    return df,y

In [10]:
X,y = clean_car_data(train_data)

In [11]:
print(pd.DataFrame(X))

                 0       1         2    3      4       5       6    7    8   \
0     580881.772727  2014.0  145500.0  8.0  23.40  1248.0   74.00  5.0  0.0   
1     490000.000000  2014.0  120000.0  4.0  21.14  1498.0  103.52  5.0  0.0   
2     167666.666667  2006.0  140000.0  3.0  17.70  1497.0   78.00  5.0  0.0   
3     225000.000000  2010.0  127000.0  8.0  23.00  1396.0   90.00  5.0  0.0   
4     130000.000000  2007.0  120000.0  8.0  16.10  1298.0   88.20  5.0  0.0   
...             ...     ...       ...  ...    ...     ...     ...  ...  ...   
6496  610000.000000  2015.0   33000.0  8.0  20.73  1373.0   91.10  5.0  0.0   
6497  979000.000000  2015.0   50000.0  8.0  19.67  1582.0  126.20  5.0  0.0   
6498  475571.285714  2012.0  120000.0  4.0  12.05  2179.0  120.00  9.0  0.0   
6499  527500.000000  2018.0   35000.0  8.0  25.40  1186.0   73.97  5.0  0.0   
6500  700000.000000  2018.0   43000.0  8.0  27.40  1498.0   98.63  5.0  0.0   

       9    10   11   12   13      14      15      

In [11]:
Xt = X.transpose()
theta = np.dot(np.linalg.inv(np.dot(Xt,X)),np.dot(Xt,y))

In [12]:
theta = theta.reshape(18,1)
theta.shape

(18, 1)

In [13]:
y = y.reshape(6501,1)

In [14]:
X,y = clean_car_data(test_data)

In [15]:
y = y.reshape(1628,1)

In [16]:
diff = np.absolute(np.dot(X,theta)-y)

In [18]:
MAE = np.nanmean(diff,axis=0)

In [19]:
print(MAE)

[77395.26095081]
